In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from docplex.cp.model import  CpoModel
from utils import extract_solution

![alternatvie text](pics/2023-07-07-09-59-58.png)  

In [2]:
arcs = {(9,0):7 , (9,1):2 , (9,2):1 ,
       (0,3):2, (0,4):4,
       (1,4):5, (1,5):6, 
       (2,3):4, (2,7):8,
       (3,6):7, (3,7):1,
       (4,6):3, (4,5):8, (4,8):3,
       (5,8):3, 
       (6,10):1,
       (7,10):3,
       (8,10):4
       }

In [3]:
all = []
for i in arcs:
    all.append([i[0],i[1],arcs[i]])
Network = pd.DataFrame(all, columns = ['origin','destination','capacity'])
Network

,origin,destination,capacity
0,9,0,7
1,9,1,2
2,9,2,1
3,0,3,2
4,0,4,4
5,1,4,5
6,1,5,6
7,2,3,4
8,2,7,8
9,3,6,7


In [4]:
mdl = CpoModel(name="NetworkFlow")
Network['flow'] = 0

for row in Network.itertuples():
    dv = mdl.integer_var(min=0, max=row.capacity, name='flow_%s_%s' %(row.origin, row.destination))
    Network['flow'].loc[row.Index]= dv

Network

,origin,destination,capacity,flow
0,9,0,7,flow_9_0 = intVar(0..7)
1,9,1,2,flow_9_1 = intVar(0..2)
2,9,2,1,"flow_9_2 = intVar(0, 1)"
3,0,3,2,flow_0_3 = intVar(0..2)
4,0,4,4,flow_0_4 = intVar(0..4)
5,1,4,5,flow_1_4 = intVar(0..5)
6,1,5,6,flow_1_5 = intVar(0..6)
7,2,3,4,flow_2_3 = intVar(0..4)
8,2,7,8,flow_2_7 = intVar(0..8)
9,3,6,7,flow_3_6 = intVar(0..7)


In [5]:
S = mdl.integer_var(min=0, max=100, name = 'Source')

In [6]:
Outgoing_Flow = Network[['origin','flow']].groupby('origin').agg(sum).reset_index().rename(columns = {'flow':"outgoing_flow"})
Outgoing_Flow

,origin,outgoing_flow
0,0,flow_0_3 + flow_0_4
1,1,flow_1_4 + flow_1_5
2,2,flow_2_3 + flow_2_7
3,3,flow_3_6 + flow_3_7
4,4,flow_4_6 + flow_4_5 + flow_4_8
5,5,flow_5_8 = intVar(0..3)
6,6,"flow_6_10 = intVar(0, 1)"
7,7,flow_7_10 = intVar(0..3)
8,8,flow_8_10 = intVar(0..4)
9,9,flow_9_0 + flow_9_1 + flow_9_2


In [7]:
Incoming_Flow = Network[['destination','flow']].groupby('destination').agg(sum).reset_index().rename(columns = {'flow':"incoming_Flow"})
Incoming_Flow

,destination,incoming_Flow
0,0,flow_9_0 = intVar(0..7)
1,1,flow_9_1 = intVar(0..2)
2,2,"flow_9_2 = intVar(0, 1)"
3,3,flow_0_3 + flow_2_3
4,4,flow_0_4 + flow_1_4
5,5,flow_1_5 + flow_4_5
6,6,flow_3_6 + flow_4_6
7,7,flow_2_7 + flow_3_7
8,8,flow_4_8 + flow_5_8
9,10,flow_6_10 + flow_7_10 + flow_8_10


In [8]:
Inflow_Outflow = pd.merge(Incoming_Flow, Outgoing_Flow, left_on='destination', right_on = 'origin', how = 'outer')
Inflow_Outflow

,destination,incoming_Flow,origin,outgoing_flow
0,0.0,flow_9_0 = intVar(0..7),0.0,flow_0_3 + flow_0_4
1,1.0,flow_9_1 = intVar(0..2),1.0,flow_1_4 + flow_1_5
2,2.0,"flow_9_2 = intVar(0, 1)",2.0,flow_2_3 + flow_2_7
3,3.0,flow_0_3 + flow_2_3,3.0,flow_3_6 + flow_3_7
4,4.0,flow_0_4 + flow_1_4,4.0,flow_4_6 + flow_4_5 + flow_4_8
5,5.0,flow_1_5 + flow_4_5,5.0,flow_5_8 = intVar(0..3)
6,6.0,flow_3_6 + flow_4_6,6.0,"flow_6_10 = intVar(0, 1)"
7,7.0,flow_2_7 + flow_3_7,7.0,flow_7_10 = intVar(0..3)
8,8.0,flow_4_8 + flow_5_8,8.0,flow_8_10 = intVar(0..4)
9,10.0,flow_6_10 + flow_7_10 + flow_8_10,NaN,NaN


In [9]:
Inflow_Outflow.loc[Inflow_Outflow['origin']==9,'incoming_Flow']=S

Inflow_Outflow.loc[Inflow_Outflow['destination']==10,'outgoing_flow']=S

In [10]:
Inflow_Outflow

,destination,incoming_Flow,origin,outgoing_flow
0,0.0,flow_9_0 = intVar(0..7),0.0,flow_0_3 + flow_0_4
1,1.0,flow_9_1 = intVar(0..2),1.0,flow_1_4 + flow_1_5
2,2.0,"flow_9_2 = intVar(0, 1)",2.0,flow_2_3 + flow_2_7
3,3.0,flow_0_3 + flow_2_3,3.0,flow_3_6 + flow_3_7
4,4.0,flow_0_4 + flow_1_4,4.0,flow_4_6 + flow_4_5 + flow_4_8
5,5.0,flow_1_5 + flow_4_5,5.0,flow_5_8 = intVar(0..3)
6,6.0,flow_3_6 + flow_4_6,6.0,"flow_6_10 = intVar(0, 1)"
7,7.0,flow_2_7 + flow_3_7,7.0,flow_7_10 = intVar(0..3)
8,8.0,flow_4_8 + flow_5_8,8.0,flow_8_10 = intVar(0..4)
9,10.0,flow_6_10 + flow_7_10 + flow_8_10,NaN,Source = intVar(0..100)


In [11]:
for row in Inflow_Outflow.itertuples():
    mdl.add( row.incoming_Flow == row.outgoing_flow )

In [12]:
obj = mdl.maximize ( S )
mdl.add(obj)

msol = mdl.solve(TimeLimit=10)
msol.print_solution()

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Maximization problem - 19 variables, 11 constraints
 ! Presolve      : 8 extractables eliminated
 ! TimeLimit            = 10
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 31.4 (before), 31.4 (after)
 !  . Memory usage      : 300.2 kB (before), 300.2 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         19                 -
 + New bound is 7
 *             7        4  0.02s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 7 (optimal - effective tol. is 0)
 ! Best bound             : 7
 ! ----------------------------------------------------------------------

In [13]:
extract_solution(Inflow_Outflow,mdl, msol, drop = False )

,destination,incoming_Flow,origin,outgoing_flow,incoming_Flow_Solution,outgoing_flow_Solution
0,0.0,flow_9_0 = intVar(0..7),0.0,flow_0_3 + flow_0_4,4,4
1,1.0,flow_9_1 = intVar(0..2),1.0,flow_1_4 + flow_1_5,2,2
2,2.0,"flow_9_2 = intVar(0, 1)",2.0,flow_2_3 + flow_2_7,1,1
3,3.0,flow_0_3 + flow_2_3,3.0,flow_3_6 + flow_3_7,1,1
4,4.0,flow_0_4 + flow_1_4,4.0,flow_4_6 + flow_4_5 + flow_4_8,5,5
5,5.0,flow_1_5 + flow_4_5,5.0,flow_5_8 = intVar(0..3),3,3
6,6.0,flow_3_6 + flow_4_6,6.0,"flow_6_10 = intVar(0, 1)",1,1
7,7.0,flow_2_7 + flow_3_7,7.0,flow_7_10 = intVar(0..3),2,2
8,8.0,flow_4_8 + flow_5_8,8.0,flow_8_10 = intVar(0..4),4,4
9,10.0,flow_6_10 + flow_7_10 + flow_8_10,nan,Source = intVar(0..100),7,7
